In [1]:
import pandas as pd #the csv reading library
import matplotlib.pyplot as plt #the ploting library
import glob #to get file names
import numpy as np
from pandas import Series
import seaborn as sns
from matplotlib.pyplot import figure
import cmocean.cm as cmo
import seaborn as sns

from pylab import rcParams
from numpy import diff
from scipy import interpolate
import scipy.stats as stats

from scipy.signal import savgol_filter
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols

import matplotlib.gridspec as gridspec

import cmocean
import cmocean.cm as cmo
import seaborn as sns


import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [2]:
df_iron = pd.read_csv('../Dissertation_projects/Data/IRON/Iron.csv', sep=',', index_col=0)

#make all the spots labeled NA a nan value
df_iron = df_iron.replace('NA', np.nan)
df_iron = df_iron.replace('NA ', np.nan)

#make all BDL a 0 
df_iron.loc[df_iron['FeT_uM'] == 'BLD', 'FeT_uM'] = 0
df_iron.loc[df_iron['Fe2_uM'] == 'BLD', 'Fe2_uM'] = 0

# convert columns to numeric data type
df_iron['FeT_uM'] = pd.to_numeric(df_iron['FeT_uM'], errors='coerce')
df_iron['Fe2_uM'] = pd.to_numeric(df_iron['Fe2_uM'], errors='coerce')

df_iron['FeT_uM'] = df_iron['FeT_uM'].where(pd.notna(df_iron['FeT_uM']), other=np.nan)
df_iron['Fe2_uM'] = df_iron['Fe2_uM'].where(pd.notna(df_iron['Fe2_uM']), other=np.nan)

#make all negative a 0 
df_iron.loc[df_iron['FeT_uM'] < 0, 'FeT_uM'] = 0
df_iron.loc[df_iron['Fe2_uM'] < 0, 'Fe2_uM'] = 0

#calculate Fe +3 
df_iron['Fe3_uM'] = df_iron['FeT_uM'] - df_iron['Fe2_uM']
df_iron.loc[df_iron['Fe3_uM'] < 0, 'Fe3_uM'] = 0
df_iron.loc[df_iron['Fe2_uM'] > df_iron['FeT_uM'], 'Fe3_uM'] = 0
df_iron['Fe3_uM'] = pd.to_numeric(df_iron['Fe3_uM'], errors='coerce')
df_iron['Fe3_uM'] = df_iron['Fe3_uM'].where(pd.notna(df_iron['Fe3_uM']), other=np.nan)

#time feild 
df_iron['Fe2_Time'] = pd.to_numeric(df_iron['Fe2_Time'], errors='coerce')
df_iron['Fe2_Time'] = df_iron['Fe2_Time'].where(pd.notna(df_iron['Fe2_Time']), other=np.nan)

#time feild 
df_iron['FeT_Time'] = pd.to_numeric(df_iron['FeT_Time'], errors='coerce')
df_iron['FeT_Time'] = df_iron['FeT_Time'].where(pd.notna(df_iron['FeT_Time']), other=np.nan)

df_iron['code'] = df_iron['Location'] + '_' + df_iron['Campaign']

In [3]:
df_doc = pd.read_csv('../Dissertation_projects/Data/TOC/TOC_GOM.csv', sep=',', index_col=0)

#make all BDL a 0 
df_doc.loc[df_doc['NPOC_uM'] == 'BLD', 'NPOC_uM'] = 0
df_doc.loc[df_doc['TN_uM'] == 'BLD', 'TN_uM'] = 0

df_doc.loc[df_doc['NPOC_uM'] < 0, 'NPOC_uM'] = 0
df_doc.loc[df_doc['TN_uM'] < 0, 'TN_uM'] = 0

#fix date time 
df_doc['date_time'] = pd.to_datetime(df_doc['Date'])
df_doc['Date'] = df_doc['date_time'].dt.date

df_doc['NPOC_uM'] = pd.to_numeric(df_doc['NPOC_uM'], errors='coerce')
df_doc['NPOC_uM'] = df_doc['NPOC_uM'].where(pd.notna(df_doc['NPOC_uM']), other=np.nan)

df_doc['TN_uM'] = pd.to_numeric(df_doc['TN_uM'], errors='coerce')
df_doc['TN_uM'] = df_doc['TN_uM'].where(pd.notna(df_doc['TN_uM']), other=np.nan)

df_doc['Time'] = pd.to_numeric(df_doc['Time'], errors='coerce')
df_doc['Time'] = df_doc['Time'].where(pd.notna(df_doc['Time']), other=np.nan)

df_doc['code'] = df_doc['Location'] + '_' + df_doc['Campaign']

In [4]:
df_nisk = pd.read_csv('../Dissertation_projects/Data/CTD/NISK_SUMS.csv', sep=',', index_col=0)

# drop rows with NaN index values
df_nisk = df_nisk[df_nisk['Sample_ID'].notna()]
df_nisk['code'] = df_nisk['station'] + '_' + df_nisk['camp']

# reformat the index to be a whole number
df_nisk.index = df_nisk.index.astype(int)

In [5]:
df_meta = pd.read_csv('../Dissertation_projects/Data/meta_data.csv', sep=',', index_col=0)

#fix date time 
df_meta['date_time'] = pd.to_datetime(df_meta['Date'])
df_meta['Date'] = df_meta['date_time'].dt.date

df_meta['code'] = df_meta['Station'] + '_' + df_meta['Trip']

df_meta

,Date,Trip,Station,Lat,Lon,Water_depth,Sample_depth,Sample_depth_units,Wavguide_Path_lengths,BC_time,BC_Chamber_vol_(L),Type,Waveguide_used,Notes,Flagged,Salinity,date_time,code
Sample,,,,,,,,,,,,,,,,,,
211901,2021-07-15,GOM2021,St.MK,28.906820,-90.333160,16.7,16.7,M,NaN,NaN,NaN,NISK,NaN,NaN,NaN,35.223851,2021-07-15,St.MK_GOM2021
211902,2021-07-15,GOM2021,St.MK,28.906820,-90.333163,16.7,16.7,M,NaN,NaN,NaN,NISK,NaN,NaN,NaN,35.183855,2021-07-15,St.MK_GOM2021
211903,2021-07-15,GOM2021,St.MK,28.906660,-90.334096,16.7,14,M,NaN,NaN,NaN,NISK,NaN,NaN,NaN,34.505918,2021-07-15,St.MK_GOM2021
211904,2021-07-15,GOM2021,St.MK,28.906660,-90.334119,16.7,14,M,NaN,NaN,NaN,NISK,NaN,NaN,NaN,34.486418,2021-07-15,St.MK_GOM2021
211905,2021-07-15,GOM2021,St.MK,28.906531,-90.334841,16.7,10,M,NaN,NaN,NaN,NISK,NaN,NaN,NaN,32.824471,2021-07-15,St.MK_GOM2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225390,2022-07-23,GOM22SU,St.MK,NaN,NaN,NaN,30,CM,0.01,NaN,NaN,RHZ,1CM CUV,NaN,NaN,36.007597,2022-07-23,St.MK_GOM22SU
225391,2022-07-23,GOM22SU,St.MK,NaN,NaN,NaN,34,CM,0.01,NaN,NaN,RHZ,1CM CUV,NaN,NaN,36.007597,2022-07-23,St.MK_GOM22SU
225392,2022-07-23,GOM22SU,St.MK,NaN,NaN,NaN,38,CM,0.01,NaN,NaN,RHZ,1CM CUV,NaN,NaN,36.007597,2022-07-23,St.MK_GOM22SU


In [6]:
df_c = pd.read_csv('../Dissertation_projects/Data/CDOM/Outputs.csv', sep=',', index_col=0)

df_c['Sample_idx'] = df_c.index.astype(object)
df_meta['Sample_idx'] = df_meta.index.astype(object)

df_c.index = df_c['file_id']

df_cdom = df_c.merge(df_meta, on="Sample_idx", how="left")

df_cdom.index = df_cdom['Sample_idx']
df_cdom['Sample'] = df_cdom.index
df_cdom.index = df_cdom['Sample']
df_cdom = df_cdom.drop('Sample_idx', axis = 1)
df_meta = df_meta.drop('Sample_idx', axis = 1)

In [7]:
#get BC data sets 
df_cdom_bc = df_cdom[df_cdom['Type'] == 'LANDSYR']
df_iron_bc = df_iron[df_iron['Type'] == 'BC']
df_doc_bc = df_doc[df_doc['Type'] == 'BC']

#get PW data sets 
df_cdom_pw = df_cdom[df_cdom['Type'] == 'PW']
df_iron_pw = df_iron[df_iron['Type'] == 'PW']
df_doc_pw = df_doc[df_doc['Type'] == 'PW']

#get WC data sets 
df_cdom_wc = df_cdom[df_cdom['Type'] == 'NISK']
df_iron_wc = df_iron[df_iron['Type'] == 'WC']
df_doc_wc = df_doc[df_doc['Type'] == 'WC']

#get RHZ data 
df_cdom_rz = df_cdom[df_cdom['Type'] == 'RHZ']
df_iron_rz = df_iron[df_iron['Type'].isin(['RZ', 'GBRZ'])]
df_doc_rz = df_doc[df_doc['Type'] == 'RZ']

In [8]:
df_cdom_bc['Sample_idx'] = df_cdom_bc.index.astype(object)
df_doc_bc['Sample_idx'] = df_doc_bc.index.astype(object)

df_cdom_bc.index = df_cdom_bc['file_id']

df_cdom_doc = df_cdom_bc.merge(df_doc_bc, on="Sample_idx", how="left")

df_cdom_doc.index = df_cdom_doc['Sample_idx']
df_cdom_doc['Sample'] = df_cdom_doc.index
df_cdom_doc.index = df_cdom_doc['Sample']

df_cdom_bc.index = df_cdom_bc['Sample_idx']
df_cdom_bc['Sample'] = df_cdom_bc.index
df_cdom_bc.index = df_cdom_bc['Sample']

df_doc_bc.index = df_doc_bc['Sample_idx']
df_doc_bc['Sample'] = df_doc_bc.index
df_doc_bc.index = df_doc_bc['Sample']

df_cdom_doc = df_cdom_doc.drop('Sample_idx', axis = 1)
df_doc_bc = df_doc_bc.drop('Sample_idx', axis = 1)
df_cdom_bc = df_cdom_bc.drop('Sample_idx', axis = 1)


# Step 2: Conducting a linear regression
df_cdom_doc = df_cdom_doc.dropna(subset=['NPOC_uM', 'acdom_350_bc'])
df_cdom_doc = df_cdom_doc.drop(df_cdom_doc[(df_cdom_doc['NPOC_uM'] == 0) | (df_cdom_doc['acdom_350_bc'] >2)].index)

slope, intercept, r_value, p_value, std_err = stats.linregress(df_cdom_doc['acdom_350_bc'], df_cdom_doc['NPOC_uM'])

# Step 3: Finding the ratio where NPOC_uM is equal to acdom_350_bc
# slope * x + intercept = x  =>  x * (slope - 1) = -intercept  =>  x = -intercept / (slope - 1)
x_equal = slope
x_err = std_err


# Plotting
plt.figure(figsize=(10, 6))
# Step 1: Plotting the scatter plot
plt.scatter(df_cdom_doc['acdom_350_bc'], df_cdom_doc['NPOC_uM'], color='blue', label='Data points')
# Step 4: Plotting the regression line
x_vals = df_cdom_doc['acdom_350_bc']
y_vals = slope * x_vals + intercept
plt.plot(x_vals, y_vals, color='red', label=f'Regression Line (y={slope:.2f}x + {intercept:.2f})')

plt.xlim(0,2)
plt.ylim(0,400)

# Step 5: Annotating the plot with the calculated ratio and r-value
plt.annotate(f'slope (NPOC:acdom) = {x_equal:.2f}', xy=(0.07, 0.9), xycoords='axes fraction')
plt.annotate(f'R-value = {r_value:.2f}', xy=(0.1, 0.85), xycoords='axes fraction')

plt.ylabel('NPOC_uM')
plt.xlabel('acdom_350_bc')
plt.legend()
plt.title('Scatter plot with Linear Regression')
plt.grid(True)
plt.show()

x_err

C:\Users\hmlbr\AppData\Local\Temp\ipykernel_12332\3227746585.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_doc_bc['Sample_idx'] = df_doc_bc.index.astype(object)
C:\Users\hmlbr\AppData\Local\Temp\ipykernel_12332\3227746585.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_doc_bc['Sample'] = df_doc_bc.index


ValueError: Inputs must not be empty.

df_cdom_bc['Sample_idx'] = df_cdom_bc.index.astype(object)
df_iron_bc['Sample_idx'] = df_iron_bc.index.astype(object)

df_cdom_bc.index = df_cdom_bc['file_id']

df_fe_cdom = df_cdom_bc.merge(df_iron_bc, on="Sample_idx", how="left")

df_fe_cdom.index = df_fe_cdom['Sample_idx']
df_fe_cdom['Sample'] = df_fe_cdom.index
df_fe_cdom.index = df_fe_cdom['Sample']

df_cdom_bc.index = df_cdom_bc['Sample_idx']
df_cdom_bc['Sample'] = df_cdom_bc.index
df_cdom_bc.index = df_cdom_bc['Sample']

df_iron_bc.index = df_iron_bc['Sample_idx']
df_iron_bc['Sample'] = df_iron_bc.index
df_iron_bc.index = df_iron_bc['Sample']

df_fe_cdom = df_fe_cdom.drop('Sample_idx', axis = 1)
df_iron_bc = df_iron_bc.drop('Sample_idx', axis = 1)
df_cdom_bc = df_cdom_bc.drop('Sample_idx', axis = 1)


# Step 2: Conducting a linear regression
df_fe_cdom = df_fe_cdom.dropna(subset=['FeT_uM', 'acdom_350_bc'])
#df_fe_cdom = df_fe_cdom.drop(df_fe_cdom[(df_fe_cdom['FeT_uM'] == 0) | (df_fe_cdom['acdom_350_bc'] >2)].index)

slope, intercept, r_value, p_value, std_err = stats.linregress(df_fe_cdom['acdom_350_bc'], df_fe_cdom['FeT_uM'])

# Step 3: Finding the ratio where NPOC_uM is equal to acdom_350_bc
# slope * x + intercept = x  =>  x * (slope - 1) = -intercept  =>  x = -intercept / (slope - 1)
x_equal = slope
x_err = std_err


# Plotting
plt.figure(figsize=(10, 6))
# Step 1: Plotting the scatter plot
plt.scatter(df_fe_cdom['acdom_350_bc'], df_fe_cdom['FeT_uM'], color='blue', label='Data points')
# Step 4: Plotting the regression line
x_vals = df_fe_cdom['acdom_350_bc']
y_vals = slope * x_vals + intercept
plt.plot(x_vals, y_vals, color='red', label=f'Regression Line (y={slope:.2f}x + {intercept:.2f})')

plt.xlim(0,2)
plt.ylim(0,400)

# Step 5: Annotating the plot with the calculated ratio and r-value
plt.annotate(f'slope (NPOC:acdom) = {x_equal:.2f}', xy=(0.07, 0.9), xycoords='axes fraction')
plt.annotate(f'R-value = {r_value:.2f}', xy=(0.1, 0.85), xycoords='axes fraction')

plt.ylabel('NPOC_uM')
plt.xlabel('acdom_350_bc')
plt.legend()
plt.title('Scatter plot with Linear Regression')
plt.grid(True)
plt.show()

x_err

In [ ]:
#get BC data sets 
df_cdom_bc = df_cdom[df_cdom['Type'] == 'LANDSYR']
df_iron_bc = df_iron[df_iron['Type'] == 'BC']
df_doc_bc = df_doc[df_doc['Type'] == 'BC']

#get PW data sets 
df_cdom_pw = df_cdom[df_cdom['Type'] == 'PW']
df_iron_pw = df_iron[df_iron['Type'] == 'PW']
df_doc_pw = df_doc[df_doc['Type'] == 'PW']

#get WC data sets 
df_cdom_wc = df_cdom[df_cdom['Type'] == 'NISK']
df_iron_wc = df_iron[df_iron['Type'] == 'WC']
df_doc_wc = df_doc[df_doc['Type'] == 'WC']

#get RHZ data 
df_cdom_rz = df_cdom[df_cdom['Type'] == 'RHZ']
df_iron_rz = df_iron[df_iron['Type'].isin(['RZ', 'GBRZ'])]
df_doc_rz = df_doc[df_doc['Type'] == 'RZ']

In [ ]:
df_hights = pd.read_csv('../Dissertation_projects/Data/Hights.csv', sep=',')

df_hights['code'] = df_hights['Stn'] + '_' + df_hights['Camp']
#df_hights.drop("Stn", axis=1, inplace=True)
#df_hights.drop("Camp", axis=1, inplace=True)
df_hights.index = df_hights['code']
df_hights.drop("code", axis=1, inplace=True)

In [ ]:
#DOC slopes 
df_doc_bc_plot = df_doc_bc[(df_doc_bc['NPOC_uM'] != 0)]
df_doc_bc_plot = df_doc_bc_plot.dropna(subset=["NPOC_uM"])
codes = df_doc_bc_plot['code'].unique()
df_doc_bc_plot = df_doc_bc_plot[df_doc_bc_plot['code'] != 'St.13_GOM22SP']

#make plot
fig, axs = plt.subplots(nrows=10, ncols=3, figsize=(14, 20)) 

# Group the data by 'code' and iterate over each subgroup
for i, (code, group) in enumerate(df_doc_bc_plot.groupby('code')):
    # Extract x, y, and c data from the group
    
    x = group['Time']
    y = group['NPOC_uM']
    
    xx4 = group.loc[group['Time'] < 4, 'Time'].ravel()
    yy4 = group.loc[group['Time'] < 4, 'NPOC_uM'].ravel()
    
    xx6 = group.loc[group['Time'] < 6, 'Time'].ravel()
    yy6 = group.loc[group['Time'] < 6, 'NPOC_uM'].ravel()
    
    xx8 = group.loc[group['Time'] < 8, 'Time'].ravel()
    yy8 = group.loc[group['Time'] < 8, 'NPOC_uM'].ravel()
    
    xx10 = group.loc[group['Time'] < 10, 'Time'].ravel()
    yy10 = group.loc[group['Time'] < 10, 'NPOC_uM'].ravel()
    
    # Plot the data for this subgroup on its corresponding subplot
    ax = axs[i // 3, i % 3]
    sc = ax.scatter(x, y, c="black")
    
    # Add subplot labels
    ax.set_title(code)
    ax.set_xlabel('Time in Hours')
    ax.set_ylabel('DOC uM')
    ax.set_xlim(0,df_doc_bc['Time'].max())
    ax.set_ylim(df_doc_bc['NPOC_uM'].min(),df_doc_bc['NPOC_uM'].max())
    
    # Calculate the slope of the regression line
    slope4, intercept4, r_value4, p_value4, std_err4 = stats.linregress(xx4, yy4)
    slope6, intercept6, r_value6, p_value6, std_err6 = stats.linregress(xx6, yy6)
    slope8, intercept8, r_value8, p_value8, std_err6 = stats.linregress(xx8, yy8)
    slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(xx10, yy10)

    # Add the regression line to the plot
    ax.plot(x, intercept4 + slope4 * x, c='r')
    ax.plot(x, intercept6 + slope6 * x, c='g')
    ax.plot(x, intercept8 + slope8 * x, c='b')
    ax.plot(x, intercept1 + slope1 * x, c='y')

    
    # Add the equation of the regression line to the plot
    #equation = f'y = {slope:.2f}x + {intercept:.2f}'
    #ax.text(1, 390, equation)
    
    # Add the r-squared value to the plot
    rsquared4 = f'R² = {r_value4**2:.2f}'
    rsquared6 = f'R² = {r_value6**2:.2f}'
    rsquared8 = f'R² = {r_value8**2:.2f}'
    rsquared1 = f'R² = {r_value1**2:.2f}'

    ax.text(1, 400, rsquared4, c='r')
    ax.text(1, 300, rsquared6, c='g')
    ax.text(1, 200, rsquared8, c='b')
    ax.text(1, 100, rsquared1, c='y')
    
    # Add the slope value to the plot
    #sloped = f'Slope = {slope:.3f}'
    #ax.text(1, 270, sloped, fontweight='bold')

# Add overall figure labels and color bar
fig.suptitle(r'DOC Slopes and intercepts' '\n')
plt.tight_layout()
fig.set_facecolor('w')

plt.savefig("../Dissertation_projects/Figures/Flux_figures/DOC_flux.png")

plt.show()


In [ ]:
#DOC slopes Calculate 
Frames = []

# Group the data by 'code' and iterate over each subgroup
for i, (code, group) in enumerate(df_doc_bc_plot.groupby('code')):
    # Extract x, y, and c data from the group
    
    xx4 = group.loc[group['Time'] < 4, 'Time'].ravel()
    yy4 = group.loc[group['Time'] < 4, 'NPOC_uM'].ravel()
    
    xx6 = group.loc[group['Time'] < 6, 'Time'].ravel()
    yy6 = group.loc[group['Time'] < 6, 'NPOC_uM'].ravel()
    
    xx8 = group.loc[group['Time'] < 8, 'Time'].ravel()
    yy8 = group.loc[group['Time'] < 8, 'NPOC_uM'].ravel()
    
    xx10 = group.loc[group['Time'] < 10, 'Time'].ravel()
    yy10 = group.loc[group['Time'] < 10, 'NPOC_uM'].ravel()    
        
    # Calculate the slope of the regression line
    slope4, intercept4, r_value4, p_value4, std_err4 = stats.linregress(xx4, yy4)
    slope6, intercept6, r_value6, p_value6, std_err6 = stats.linregress(xx6, yy6)
    slope8, intercept8, r_value8, p_value8, std_err8 = stats.linregress(xx8, yy8)
    slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(xx10, yy10)
    
    r_squared_values = {
        'r4': std_err4,
        'r6': std_err6,
        'r8': std_err8,
        'r10':std_err1}

    r_squared_values = {key: value for key, value in r_squared_values.items() if value != 0}
    max_key = min(r_squared_values, key=r_squared_values.get)

    if max_key == 'r4':
        best_slope = slope4
        best_r_squared = r_value4**2
        best_std_err = std_err4
        note_hours = "4 hours"
    elif max_key == 'r6':
        best_slope = slope6
        best_r_squared = r_value6**2
        best_std_err = std_err6
        note_hours = "6 hours"
    elif max_key == 'r8':
        best_slope = slope8
        best_r_squared = r_value8**2
        best_std_err = std_err8 
        note_hours = "8 hours"
    else:
        best_slope = slope1
        best_r_squared = r_value1**2
        best_std_err = std_err1
        note_hours = "10 hours"
    
    # Add the slope value to the plot
    slope = best_slope
    rsquared = best_r_squared
    std_err = best_std_err
    
    dic = {'code':[code],
           'slope_doc':[slope],
           'rsquared_doc':[rsquared], 
           'std_err_doc':[std_err],
           'note_doc': [note_hours]}
    
    slopes = pd.DataFrame(dic)
    slopes = slopes.set_index('code', drop=True)
    
    Frames.append(slopes)

slopes_doc = pd.concat(Frames, axis=0, ignore_index=False)
slopes_doc

In [ ]:
#Nitrogen slopes
fig, axs = plt.subplots(nrows=10, ncols=3, figsize=(16, 20)) 

df_tn_bc_plot = df_doc_bc[(df_doc_bc['TN_uM'] != 0)]
df_tn_bc_plot = df_tn_bc_plot.dropna(subset=["TN_uM"])
codes = df_tn_bc_plot['code'].unique()
df_tn_bc_plot = df_tn_bc_plot[df_tn_bc_plot['code'] != 'St.13_GOM22SP']

# Group the data by 'code' and iterate over each subgroup
for i, (code, group) in enumerate(df_tn_bc_plot.groupby('code')):
    # Extract x, y, and c data from the group
    
    x = group['Time']
    y = group['TN_uM']
    
    xx4 = group.loc[group['Time'] < 4, 'Time'].ravel()
    yy4 = group.loc[group['Time'] < 4, 'TN_uM'].ravel()
    
    xx6 = group.loc[group['Time'] < 6, 'Time'].ravel()
    yy6 = group.loc[group['Time'] < 6, 'TN_uM'].ravel()
    
    xx8 = group.loc[group['Time'] < 8, 'Time'].ravel()
    yy8 = group.loc[group['Time'] < 8, 'TN_uM'].ravel()
    
    xx10 = group.loc[group['Time'] < 10, 'Time'].ravel()
    yy10 = group.loc[group['Time'] < 10, 'TN_uM'].ravel()
    
    # Plot the data for this subgroup on its corresponding subplot
    ax = axs[i // 3, i % 3]
    sc = ax.scatter(x, y, c='green')
    
    # Add subplot labels
    ax.set_title(code)
    ax.set_xlabel('Time')
    ax.set_ylabel('TN uM')
    ax.set_xlim(0,df_doc_bc['Time'].max())
    ax.set_ylim(df_doc_bc['TN_uM'].min(),df_doc_bc['TN_uM'].max())
    
    # Calculate the slope of the regression line
    slope4, intercept4, r_value4, p_value4, std_err4 = stats.linregress(xx4, yy4)
    slope6, intercept6, r_value6, p_value6, std_err6 = stats.linregress(xx6, yy6)
    slope8, intercept8, r_value8, p_value8, std_err6 = stats.linregress(xx8, yy8)
    slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(xx10, yy10)

    # Add the regression line to the plot
    ax.plot(x, intercept4 + slope4 * x, c='r')
    ax.plot(x, intercept6 + slope6 * x, c='g')
    ax.plot(x, intercept8 + slope8 * x, c='b')
    ax.plot(x, intercept1 + slope1 * x, c='y')

    
    # Add the equation of the regression line to the plot
    #equation = f'y = {slope:.2f}x + {intercept:.2f}'
    #ax.text(1, 390, equation)
    
    # Add the r-squared value to the plot
    rsquared4 = f'R² = {r_value4**2:.2f}'
    rsquared6 = f'R² = {r_value6**2:.2f}'
    rsquared8 = f'R² = {r_value8**2:.2f}'
    rsquared1 = f'R² = {r_value1**2:.2f}'

    ax.text(1, 35, rsquared4, c='r')
    ax.text(1, 30, rsquared6, c='g')
    ax.text(1, 25, rsquared8, c='b')
    ax.text(1, 20, rsquared1, c='y')
    
    # Add the slope value to the plot
    #sloped = f'Slope = {slope:.3f}'
    #ax.text(1, 270, sloped, fontweight='bold')

# Add overall figure labels and color bar
fig.suptitle(r'TN Slopes and intercepts:'+' \n')
plt.tight_layout()
fig.set_facecolor('w')

plt.savefig("../Dissertation_projects/Figures/Flux_figures/TN_flux.png")

plt.show()


In [ ]:
#TN slopes Calculate 
Frames = []

# Group the data by 'code' and iterate over each subgroup
for i, (code, group) in enumerate(df_tn_bc_plot.groupby('code')):
    # Extract x, y, and c data from the group
    
    xx4 = group.loc[group['Time'] < 4, 'Time'].ravel()
    yy4 = group.loc[group['Time'] < 4, 'TN_uM'].ravel()
    
    xx6 = group.loc[group['Time'] < 6, 'Time'].ravel()
    yy6 = group.loc[group['Time'] < 6, 'TN_uM'].ravel()
    
    xx8 = group.loc[group['Time'] < 8, 'Time'].ravel()
    yy8 = group.loc[group['Time'] < 8, 'TN_uM'].ravel()
    
    xx10 = group.loc[group['Time'] < 10, 'Time'].ravel()
    yy10 = group.loc[group['Time'] < 10, 'TN_uM'].ravel()    
        
    # Calculate the slope of the regression line
    slope4, intercept4, r_value4, p_value4, std_err4 = stats.linregress(xx4, yy4)
    slope6, intercept6, r_value6, p_value6, std_err6 = stats.linregress(xx6, yy6)
    slope8, intercept8, r_value8, p_value8, std_err8 = stats.linregress(xx8, yy8)
    slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(xx10, yy10)
    
    r_squared_values = {
        'r4': std_err4,
        'r6': std_err6,
        'r8': std_err8,
        'r10':std_err1}

    r_squared_values = {key: value for key, value in r_squared_values.items() if value != 0}
    max_key = min(r_squared_values, key=r_squared_values.get)

    if max_key == 'r4':
        best_slope = slope4
        best_r_squared = r_value4**2
        best_std_err = std_err4
        note_hours = "4 hours"
    elif max_key == 'r6':
        best_slope = slope6
        best_r_squared = r_value6**2
        best_std_err = std_err6
        note_hours = "6 hours"
    elif max_key == 'r8':
        best_slope = slope8
        best_r_squared = r_value8**2
        best_std_err = std_err8 
        note_hours = "8 hours"
    else:
        best_slope = slope1
        best_r_squared = r_value1**2
        best_std_err = std_err1
        note_hours = "10 hours"
    
    # Add the slope value to the plot
    slope = best_slope
    rsquared = best_r_squared
    std_err = best_std_err
    
    dic = {'code':[code],
           'slope_tn':[slope],
           'rsquared_tn':[rsquared], 
           'std_err_tn':[std_err],
           'note_tn': [note_hours]}
    
    slopes = pd.DataFrame(dic)
    slopes = slopes.set_index('code', drop=True)
    
    Frames.append(slopes)

slopes_tn = pd.concat(Frames, axis=0, ignore_index=False)
slopes_tn

#CDOM slopes

#df_cdom_bc_plot = df_cdom_bc[df_cdom_bc['code'] != 'St.13_GOM22SP']
#df_cdom_bc_plot = df_cdom_bc_plot[df_cdom_bc_plot['code'] != 'St.14_GOM2021']
#df_cdom_bc_plot = df_cdom_bc_plot.dropna(subset=['acdom_412_bc'])
codes = df_cdom_bc_plot['code'].unique()

fig, axs = plt.subplots(nrows=13, ncols=3, figsize=(16, 25)) 

# Group the data by 'code' and iterate over each subgroup
for i, (code, group) in enumerate(df_cdom_bc_plot.groupby('code')):
    # Extract x, y, and c data from the group
    
    x = group['BC_time']
    y = group['acdom_412_bc']
    
    xx4 = group.loc[group['BC_time'] < 4, 'BC_time'].ravel()
    yy4 = group.loc[group['BC_time'] < 4, 'acdom_412_bc'].ravel()
    
    xx6 = group.loc[group['BC_time'] < 6, 'BC_time'].ravel()
    yy6 = group.loc[group['BC_time'] < 6, 'acdom_412_bc'].ravel()
    
    xx8 = group.loc[group['BC_time'] < 8, 'BC_time'].ravel()
    yy8 = group.loc[group['BC_time'] < 8, 'acdom_412_bc'].ravel()
    
    xx10 = group.loc[group['BC_time'] < 10, 'BC_time'].ravel()
    yy10 = group.loc[group['BC_time'] < 10, 'acdom_412_bc'].ravel()
    
    # Plot the data for this subgroup on its corresponding subplot
    ax = axs[i // 3, i % 3]
    sc = ax.scatter(x, y, c='y')
    
    # Add subplot labels
    ax.set_title(code)
    ax.set_xlabel('Time')
    ax.set_ylabel('aCDOM 412')
    ax.set_xlim(0,df_doc_bc['Time'].max())
    ax.set_ylim(-1,2)
    
    # Calculate the slope of the regression line
    slope4, intercept4, r_value4, p_value4, std_err4 = stats.linregress(xx4, yy4)
    slope6, intercept6, r_value6, p_value6, std_err6 = stats.linregress(xx6, yy6)
    slope8, intercept8, r_value8, p_value8, std_err6 = stats.linregress(xx8, yy8)
    slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(xx10, yy10)

    # Add the regression line to the plot
    ax.plot(x, intercept4 + slope4 * x, c='r')
    ax.plot(x, intercept6 + slope6 * x, c='g')
    ax.plot(x, intercept8 + slope8 * x, c='b')
    ax.plot(x, intercept1 + slope1 * x, c='y')
    
    # Add the equation of the regression line to the plot
    #equation = f'y = {slope:.2f}x + {intercept:.2f}'
    #ax.text(1, 390, equation)
    
    # Add the r-squared value to the plot
    rsquared4 = f'R² = {r_value4**2:.2f}'
    rsquared6 = f'R² = {r_value6**2:.2f}'
    rsquared8 = f'R² = {r_value8**2:.2f}'
    rsquared1 = f'R² = {r_value1**2:.2f}'

    ax.text(1, 2.2, rsquared4, c='r')
    ax.text(1, 1.8, rsquared6, c='g')
    ax.text(1, 1.4, rsquared8, c='b')
    ax.text(1, 1, rsquared1, c='y')

# Add overall figure labels and color bar
fig.suptitle(r'$a_{\mathrm{CDOM}}$ (m$^{-1}$) 412 nm'+'\n')
plt.tight_layout()
fig.set_facecolor('w')

plt.savefig("../Dissertation_projects/Figures/Flux_figures/CDOM412_flux.png")

plt.show()


In [ ]:
#CDOM slopes
df_cdom_bc_plot = df_cdom_bc[df_cdom_bc['code'] != 'St.13_GOM22SP']
df_cdom_bc_plot = df_cdom_bc_plot[df_cdom_bc_plot['code'] != 'St.14_GOM2021']
df_cdom_bc_plot = df_cdom_bc_plot.dropna(subset=['acdom_350_bc'])
codes = df_cdom_bc_plot['code'].unique()

fig, axs = plt.subplots(nrows=13, ncols=3, figsize=(16, 25)) 

# Group the data by 'code' and iterate over each subgroup
for i, (code, group) in enumerate(df_cdom_bc_plot.groupby('code')):
    # Extract x, y, and c data from the group
    
    x = group['BC_time']
    y = group['acdom_350_bc']
    
    xx4 = group.loc[group['BC_time'] < 4, 'BC_time'].ravel()
    yy4 = group.loc[group['BC_time'] < 4, 'acdom_350_bc'].ravel()
    
    xx6 = group.loc[group['BC_time'] < 6, 'BC_time'].ravel()
    yy6 = group.loc[group['BC_time'] < 6, 'acdom_350_bc'].ravel()
    
    xx8 = group.loc[group['BC_time'] < 8, 'BC_time'].ravel()
    yy8 = group.loc[group['BC_time'] < 8, 'acdom_350_bc'].ravel()
    
    xx10 = group.loc[group['BC_time'] < 10, 'BC_time'].ravel()
    yy10 = group.loc[group['BC_time'] < 10, 'acdom_350_bc'].ravel()
    
    # Plot the data for this subgroup on its corresponding subplot
    ax = axs[i // 3, i % 3]
    sc = ax.scatter(x, y, c='y')
    
    # Add subplot labels
    ax.set_title(code)
    ax.set_xlabel('Time')
    ax.set_ylabel('aCDOM 350')
    ax.set_xlim(0,df_doc_bc['Time'].max())
    ax.set_ylim(-1,2)
    
    # Calculate the slope of the regression line
    slope4, intercept4, r_value4, p_value4, std_err4 = stats.linregress(xx4, yy4)
    slope6, intercept6, r_value6, p_value6, std_err6 = stats.linregress(xx6, yy6)
    slope8, intercept8, r_value8, p_value8, std_err6 = stats.linregress(xx8, yy8)
    slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(xx10, yy10)

    # Add the regression line to the plot
    ax.plot(x, intercept4 + slope4 * x, c='r')
    ax.plot(x, intercept6 + slope6 * x, c='g')
    ax.plot(x, intercept8 + slope8 * x, c='b')
    ax.plot(x, intercept1 + slope1 * x, c='y')
    
    # Add the equation of the regression line to the plot
    #equation = f'y = {slope:.2f}x + {intercept:.2f}'
    #ax.text(1, 390, equation)
    
    # Add the r-squared value to the plot
    rsquared4 = f'R² = {r_value4**2:.2f}'
    rsquared6 = f'R² = {r_value6**2:.2f}'
    rsquared8 = f'R² = {r_value8**2:.2f}'
    rsquared1 = f'R² = {r_value1**2:.2f}'

    ax.text(1, 2.2, rsquared4, c='r')
    ax.text(1, 1.8, rsquared6, c='g')
    ax.text(1, 1.4, rsquared8, c='b')
    ax.text(1, 1, rsquared1, c='y')

# Add overall figure labels and color bar
fig.suptitle(r'$a_{\mathrm{CDOM}}$ (m$^{-1}$) 350 nm'+'\n')
plt.tight_layout()
fig.set_facecolor('w')

plt.savefig("../Dissertation_projects/Figures/Flux_figures/CDOM350_flux.png")

plt.show()


#cdom slopes Calculate 
Frames = []

# Group the data by 'code' and iterate over each subgroup
for i, (code, group) in enumerate(df_cdom_bc_plot.groupby('code')):
    # Extract x, y, and c data from the group
    
    xx4 = group.loc[group['BC_time'] < 4, 'BC_time'].ravel()
    yy4 = group.loc[group['BC_time'] < 4, 'acdom_412_bc'].ravel()
    
    xx6 = group.loc[group['BC_time'] < 6, 'BC_time'].ravel()
    yy6 = group.loc[group['BC_time'] < 6, 'acdom_412_bc'].ravel()
    
    xx8 = group.loc[group['BC_time'] < 8, 'BC_time'].ravel()
    yy8 = group.loc[group['BC_time'] < 8, 'acdom_412_bc'].ravel()
    
    xx10 = group.loc[group['BC_time'] < 10, 'BC_time'].ravel()
    yy10 = group.loc[group['BC_time'] < 10, 'acdom_412_bc'].ravel()    
        
    # Calculate the slope of the regression line
    slope4, intercept4, r_value4, p_value4, std_err4 = stats.linregress(xx4, yy4)
    slope6, intercept6, r_value6, p_value6, std_err6 = stats.linregress(xx6, yy6)
    slope8, intercept8, r_value8, p_value8, std_err8 = stats.linregress(xx8, yy8)
    slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(xx10, yy10)
    
    r_squared_values = {
        'r4': std_err4,
        'r6': std_err6,
        'r8': std_err8,
        'r10':std_err1}

    r_squared_values = {key: value for key, value in r_squared_values.items() if value != 0}
    max_key = min(r_squared_values, key=r_squared_values.get)

    if max_key == 'r4':
        best_slope = slope4
        best_r_squared = r_value4**2
        best_std_err = std_err4
        note_hours = "4 hours"
    elif max_key == 'r6':
        best_slope = slope6
        best_r_squared = r_value6**2
        best_std_err = std_err6
        note_hours = "6 hours"
    elif max_key == 'r8':
        best_slope = slope8
        best_r_squared = r_value8**2
        best_std_err = std_err8 
        note_hours = "8 hours"
    else:
        best_slope = slope1
        best_r_squared = r_value1**2
        best_std_err = std_err1
        note_hours = "10 hours"
    
    # Add the slope value to the plot
    slope = best_slope
    rsquared = best_r_squared
    std_err = best_std_err
    
    dic = {'code':[code],
           'slope_a412':[slope],
           'rsquared_a412':[rsquared], 
           'std_err_a412':[std_err],
           'note_a412': [note_hours]}
    
    slopes = pd.DataFrame(dic)
    slopes = slopes.set_index('code', drop=True)
    
    Frames.append(slopes)

slopes_a412 = pd.concat(Frames, axis=0, ignore_index=False)
slopes_a412

In [ ]:
#cdom slopes Calculate 
Frames = []

# Group the data by 'code' and iterate over each subgroup
for i, (code, group) in enumerate(df_cdom_bc_plot.groupby('code')):
    # Extract x, y, and c data from the group
    
    xx4 = group.loc[group['BC_time'] < 4, 'BC_time'].ravel()
    yy4 = group.loc[group['BC_time'] < 4, 'acdom_350_bc']*x_equal.ravel()
    
    xx6 = group.loc[group['BC_time'] < 6, 'BC_time'].ravel()
    yy6 = group.loc[group['BC_time'] < 6, 'acdom_350_bc']*x_equal.ravel()
    
    xx8 = group.loc[group['BC_time'] < 8, 'BC_time'].ravel()
    yy8 = group.loc[group['BC_time'] < 8, 'acdom_350_bc']*x_equal.ravel()
    
    xx10 = group.loc[group['BC_time'] < 10, 'BC_time'].ravel()
    yy10 = group.loc[group['BC_time'] < 10, 'acdom_350_bc']*x_equal.ravel()    
        
    # Calculate the slope of the regression line
    slope4, intercept4, r_value4, p_value4, std_err4 = stats.linregress(xx4, yy4)
    slope6, intercept6, r_value6, p_value6, std_err6 = stats.linregress(xx6, yy6)
    slope8, intercept8, r_value8, p_value8, std_err8 = stats.linregress(xx8, yy8)
    slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(xx10, yy10)
    
    r_squared_values = {
        'r4': std_err4,
        'r6': std_err6,
        'r8': std_err8,
        'r10':std_err1}

    r_squared_values = {key: value for key, value in r_squared_values.items() if value != 0}
    max_key = min(r_squared_values, key=r_squared_values.get)

    if max_key == 'r4':
        best_slope = slope4
        best_r_squared = r_value4**2
        best_std_err = std_err4
        note_hours = "4 hours"
    elif max_key == 'r6':
        best_slope = slope6
        best_r_squared = r_value6**2
        best_std_err = std_err6
        note_hours = "6 hours"
    elif max_key == 'r8':
        best_slope = slope8
        best_r_squared = r_value8**2
        best_std_err = std_err8 
        note_hours = "8 hours"
    else:
        best_slope = slope1
        best_r_squared = r_value1**2
        best_std_err = std_err1
        note_hours = "10 hours"
    
    # Add the slope value to the plot
    slope = best_slope
    rsquared = best_r_squared
    std_err = best_std_err
    
    dic = {'code':[code],
           'slope_a350':[slope],
           'rsquared_a350':[rsquared], 
           'std_err_a350':[std_err],
           'note_a350': [note_hours]}
    
    slopes = pd.DataFrame(dic)
    slopes = slopes.set_index('code', drop=True)
    
    Frames.append(slopes)

slopes_a350 = pd.concat(Frames, axis=0, ignore_index=False)
slopes_a350

In [ ]:
#clean out the Iron data for the slopes 
df_iron_bc_fe2_cl = df_iron_bc.drop(df_iron_bc[(df_iron_bc['Fe2_uM'] == 0) | (df_iron_bc['Fe2_uM'].isna())].index)

#clean out the Iron data for the slopes 
df_iron_bc_fe3_cl = df_iron_bc.drop(df_iron_bc[(df_iron_bc['Fe3_uM'] == 0) | (df_iron_bc['Fe3_uM'].isna())].index)

In [ ]:
#Iron 2 slopes 

fig, axs = plt.subplots(nrows=12, ncols=3, figsize=(16, 25)) 

# Group the data by 'code' and iterate over each subgroup
for i, (code, group) in enumerate(df_iron_bc_fe2_cl.groupby('code')):
    # Extract x, y, and c data from the group
    x = group['Fe2_Time']
    y = group['Fe2_uM']
    
    xx = group.loc[group['Fe2_Time'] <6 , 'Fe2_Time'].ravel()
    yy = group.loc[group['Fe2_Time'] <6, 'Fe2_uM'].ravel()
    
    # Plot the data for this subgroup on its corresponding subplot
    ax = axs[i // 3, i % 3]
    sc = ax.scatter(x, y, c='b')
    
    # Add subplot labels
    ax.set_title(code)
    ax.set_xlabel('Time')
    ax.set_ylabel('Iron(II) uM')
    ax.set_xlim(0,df_iron_bc_fe2_cl['Fe2_Time'].max())
    ax.set_ylim(0,max(y))
    
    if len(xx) > 0 and len(yy) > 0:
        # Calculate the slope of the regression line
        slope, intercept, r_value, p_value, std_err = stats.linregress(xx, yy)
        
        # Add the regression line to the plot
        ax.plot(x, intercept + slope * x, c='r')
    
        # Add the equation of the regression line to the plot
        equation = f'y = {slope:.2f}x + {intercept:.2f}'
        ax.text(1, max(y)*.85, equation)
    
        # Add the r-squared value to the plot
        rsquared = f'R² = {r_value**2:.2f}'
        ax.text(1, max(y)*.73, rsquared)
    
        # Add the slope value to the plot
        sloped = f'Slope = {slope:.3f}'
        ax.text(1, max(y)*.08, sloped, fontweight='bold')

# Add overall figure labels and color bar
fig.suptitle(r'Iorn (II) Slopes and intercepts' '\n')
plt.tight_layout()
plt.show()


In [ ]:
#Fe2 slopes Calculate 
Frames = []

# Group the data by 'code' and iterate over each subgroup
for i, (code, group) in enumerate(df_iron_bc_fe2_cl.groupby('code')):
    # Extract x, y, and c data from the group
    
    x = group.loc[group['Fe2_Time'] <6 , 'Fe2_Time'].ravel()
    y = group.loc[group['Fe2_Time'] <6, 'Fe2_uM'].ravel()

    if len(x) > 0 and len(y) > 0:
        # Calculate the slope of the regression line
        slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

        # Add the r-squared value to the plot
        rsquared = r_value**2
    
        # Add the slope value to the plot
        slope = slope
    
        dic = {'code':[code],
               'slope_fe2':[slope],
               'rsquared_fe2':[rsquared]}
    
        slopes = pd.DataFrame(dic)
        slopes = slopes.set_index('code', drop=True)
    
    Frames.append(slopes)

slopes_fe2 = pd.concat(Frames, axis=0, ignore_index=False)
slopes_fe2

In [ ]:
#Iron 3 slopes 

fig, axs = plt.subplots(nrows=9, ncols=3, figsize=(16, 20)) 

# Group the data by 'code' and iterate over each subgroup
for i, (code, group) in enumerate(df_iron_bc_fe3_cl.groupby('code')):
    # Extract x, y, and c data from the group
    x = group['FeT_Time'].ravel()
    y = group['Fe3_uM'].ravel()
    
    xx = group.loc[group['FeT_Time'] <6 , 'FeT_Time'].ravel()
    yy = group.loc[group['FeT_Time'] <6, 'Fe3_uM'].ravel()
    
    # Plot the data for this subgroup on its corresponding subplot
    ax = axs[i // 3, i % 3]
    sc = ax.scatter(x, y, c='r')
    
    # Add subplot labels
    ax.set_title(code)
    ax.set_xlabel('Time')
    ax.set_ylabel('Iron(III) uM')
    ax.set_xlim(0,df_iron_bc_fe3_cl['FeT_Time'].max())
    ax.set_ylim(0,max(y))
    
    if len(xx) > 0 and len(yy) > 0:
        # Calculate the slope of the regression line
        slope, intercept, r_value, p_value, std_err = stats.linregress(xx, yy)

    # Add the regression line to the plot
        ax.plot(x, intercept + slope * x, c='r')
    
    # Add the equation of the regression line to the plot
        equation = f'y = {slope:.2f}x + {intercept:.2f}'
        ax.text(1, max(y)*.85, equation)
    
    # Add the r-squared value to the plot
        rsquared = f'R² = {r_value**2:.2f}'
        ax.text(1, max(y)*.73, rsquared)
    
    # Add the slope value to the plot
        sloped = f'Slope = {slope:.3f}'
        ax.text(1, max(y)*.08, sloped, fontweight='bold')

# Add overall figure labels and color bar
fig.suptitle(r'Iorn (III) Slopes and intercepts' '\n')
plt.tight_layout()
plt.show()

In [ ]:
#Fe3 slopes Calculate 
Frames = []

# Group the data by 'code' and iterate over each subgroup
for i, (code, group) in enumerate(df_iron_bc_fe3_cl.groupby('code')):
    # Extract x, y, and c data from the group
    
    x = group.loc[group['FeT_Time'] <6 , 'FeT_Time'].ravel()
    y = group.loc[group['FeT_Time'] <6, 'Fe3_uM'].ravel()    
    
    
    if len(x) > 0 and len(y) > 0:
        # Calculate the slope of the regression line
        slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

    # Add the r-squared value to the plot
        rsquared = r_value**2
    
    # Add the slope value to the plot
        slope = slope
    
        dic = {'code':[code],
               'slope_fe3':[slope],
               'rsquared_fe3':[rsquared]}
        
        slopes = pd.DataFrame(dic)
        slopes = slopes.set_index('code', drop=True)
    
    Frames.append(slopes)

slopes_fe3 = pd.concat(Frames, axis=0, ignore_index=False)
slopes_fe3

In [ ]:
#make location data 
df_lat_lon = df_meta.groupby('code').agg({'Lat': 'mean', 'Lon': 'mean'}).reset_index().set_index('code')

In [ ]:
dataframes = [df_hights, df_lat_lon, slopes_a350, slopes_doc, slopes_tn, slopes_fe2, slopes_fe3]
for df in dataframes:
    if not df.index.is_unique:
        print("Duplicate indices found in dataframe:", df)


In [ ]:
#concat it all 
df_flux = pd.concat([df_hights, df_lat_lon, slopes_a350, slopes_doc, slopes_tn, slopes_fe2, slopes_fe3], axis=1, join='outer')

#calcute flux 
#df_flux["flux_aCDOM412"] = df_flux["slope_a412"] * (df_flux["Hight_m"]*(np.pi*((30/100)**2))) / (np.pi*((30/100)**2)) * 24 
df_flux["flux_aCDOM350"] = df_flux["slope_a350"] * (df_flux["Hight_m"]*(np.pi*((30/100)**2))) / (np.pi*((30/100)**2)) * 24 
df_flux["flux_doc"] = df_flux["slope_doc"] * (((df_flux["Hight_m"]*(np.pi*((30/100)**2)))*1000) / (np.pi*((30/100)**2))) * 24 /1000

df_flux["erros_aCDOM350"] = np.sqrt( ((df_flux["std_err_a350"] / df_flux["slope_a350"])**2) + (((df_flux["Hight_error"] / df_flux["Hight_m"])**2)/1000))  
df_flux["erros_doc"] = np.sqrt( (df_flux["std_err_doc"] / df_flux["slope_doc"])**2 + (df_flux["Hight_error"] / df_flux["Hight_m"])**2 )

#df_flux["flux_tn"] = df_flux["slope_tn"] * ((df_flux["Hight_m"]*(np.pi*((30/100)**2)))*1000) / (np.pi*((30/100)**2)) * 24 / 1000 
#df_flux["flux_fe2"] = df_flux["slope_fe2"] * df_flux["Hight_m"] *24
#df_flux["flux_fe3"] = df_flux["slope_fe3"] * df_flux["Hight_m"] *24

df_flux

In [ ]:
# Unique categories and subcategories
df = df_flux.dropna(subset=["flux_doc"])

categories = ['St.5B','St.4','St.MK','St.7','St.16','St.2','St.9','St.14','St.15','St.13','St.11','St.12']
subcategories = ['GOM2021', 'GOM21FA', 'GOM22SP', 'GOM22SU']

# Define custom colors
color_map = {
    'GOM2021': 'darkred',
    'GOM21FA': 'blue',
    'GOM22SP': 'green',
    'GOM22SU': 'purple'
}

# Define custom labels (if they differ from subcategory names)
label_map = {
    'GOM2021': 'Summer 2021',
    'GOM21FA': 'Fall 2021',
    'GOM22SP': 'Spring 2022',
    'GOM22SU': 'Summer 2022'
}

barWidth = 0.25

# Create figure with custom size
fig = plt.figure(figsize=(12, 10))
# Adjust the height ratios for the subplots
gs = gridspec.GridSpec(2, 1, height_ratios=[5, 1], hspace=0)  # Set hspace to 0

ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])

# Adjusted r for spacing between groups
r = np.arange(0, len(categories) * 1.5, 1.5)
positions = [r + i*barWidth for i in range(len(subcategories))]

# Adjusted code for broken axis and added spaces between the groups
for pos, subcat in zip(positions, subcategories):
    values = []
    errors = []
    for cat in categories:
        value = df[(df['Stn'] == cat) & (df['Camp'] == subcat)]['flux_doc'].values
        error = df[(df['Stn'] == cat) & (df['Camp'] == subcat)]['erros_doc'].values
        values.append(value[0] if len(value) > 0 else 0)
        errors.append(error[0] if len(error) > 0 else 0)
    
    ax1.bar(pos, [v if v > 0 else 0 for v in values], width=barWidth, color=color_map[subcat],
            label=label_map[subcat], yerr=[e if e > 0 else 0 for e in errors])
    ax2.bar(pos, [v if v < 0 else 0 for v in values], width=barWidth, color=color_map[subcat],
            label=label_map[subcat], yerr=[e if e > 0 else 0 for e in errors])
    
# Setting the limit for the break
ax1.set_ylim(0, 120)  
ax2.set_ylim(-120, 0)

# Removing spines and ticks for the broken axis effect
ax1.spines['bottom'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax1.xaxis.tick_top()
ax1.tick_params(labeltop='off')
ax2.xaxis.tick_bottom()

# Add some text for labels, title, and custom x-axis tick labels
ax2.set_xlabel('Stations: shallow to deep')
ax1.set_ylabel(r'Flux of DOC (mmol m$^{-2}$ day$^{-1}$)')
ax1.set_title('DOC flux: Benthic Chamber')
ax2.set_xticks(r + 1.5*barWidth)  # Adjust for the spaces between groups
ax2.set_xticklabels(categories)

# Add black horizontal line at y=0
ax1.axhline(0, color="black", linewidth=0.5)
ax2.axhline(0, color="black", linewidth=0.5)

handles, _ = plt.gca().get_legend_handles_labels()
labels = [label_map[subcat] for subcat in subcategories]
ax1.legend(handles[:len(subcategories)], labels)

fig.set_facecolor('w')
plt.tight_layout()

plt.savefig("../Dissertation_projects/Figures/Flux_figures/DOC_flux_barplot.png")
plt.show()


In [ ]:
# Unique categories and subcategories
df = df_flux.dropna(subset=["flux_doc"])

categories = ['St.5B','St.4','St.MK','St.7','St.16','St.2','St.9','St.14','St.15','St.13','St.11','St.12']
subcategories = ['GOM2021', 'GOM21FA', 'GOM22SP', 'GOM22SU']

# Define custom colors
color_map = {
    'GOM2021': 'darkred',
    'GOM21FA': 'blue',
    'GOM22SP': 'green',
    'GOM22SU': 'purple'
}

# Define custom labels (if they differ from subcategory names)
label_map = {
    'GOM2021': 'Summer 2021',
    'GOM21FA': 'Fall 2021',
    'GOM22SP': 'Spring 2022',
    'GOM22SU': 'Summer 2022'
}

barWidth = 0.25

# Create figure with custom size
fig = plt.figure(figsize=(10, 5))
# Adjust the height ratios for the subplots
gs = gridspec.GridSpec(2, 1, height_ratios=[5, 1], hspace=0)  # Set hspace to 0

ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])

# Adjusted r for spacing between groups
r = np.arange(0, len(categories) * 1.5, 1.5)
positions = [r + i*barWidth for i in range(len(subcategories))]

# Adjusted code for broken axis and added spaces between the groups
for pos, subcat in zip(positions, subcategories):
    values = []
    errors = []
    for cat in categories:
        value = df[(df['Stn'] == cat) & (df['Camp'] == subcat)]['flux_doc'].values
        error = df[(df['Stn'] == cat) & (df['Camp'] == subcat)]['erros_doc'].values
        values.append(value[0] if len(value) > 0 else 0)
        errors.append(error[0] if len(error) > 0 else 0)
    
    ax1.bar(pos, [v if v > 0 else 0 for v in values], width=barWidth, color=color_map[subcat],
            label=label_map[subcat])#, yerr=[e if e > 0 else 0 for e in errors])
    ax2.bar(pos, [v if v < 0 else 0 for v in values], width=barWidth, color=color_map[subcat],
            label=label_map[subcat])#, yerr=[e if e > 0 else 0 for e in errors])
    
# Setting the limit for the break
ax1.set_ylim(0, 95)  
ax2.set_ylim(-95, 0)

# Removing spines and ticks for the broken axis effect
ax1.spines['bottom'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax1.xaxis.tick_top()
ax1.tick_params(labeltop='off')
ax2.xaxis.tick_bottom()

# Add some text for labels, title, and custom x-axis tick labels
ax2.set_xlabel('Stations: shallow to deep')
ax1.set_ylabel(r'Flux of DOC (mmol m$^{-2}$ day$^{-1}$)')
ax1.set_title('DOC flux: Benthic Chamber')
ax2.set_xticks(r + 1.5*barWidth)  # Adjust for the spaces between groups
ax2.set_xticklabels(categories)

# Add black horizontal line at y=0
ax1.axhline(0, color="black", linewidth=0.5)
ax2.axhline(0, color="black", linewidth=0.5)

handles, _ = plt.gca().get_legend_handles_labels()
labels = [label_map[subcat] for subcat in subcategories]
ax1.legend(handles[:len(subcategories)], labels)

fig.set_facecolor('w')
plt.tight_layout()

plt.savefig("../Dissertation_projects/Figures/Flux_figures/_DOC_flux_barplot.png")
plt.show()


# Unique categories and subcategories
df = df_flux.dropna(subset=["flux_aCDOM412"])

categories = ['St.5B','St.4','St.MK','St.7','St.16','St.2','St.9','St.14','St.15','St.13','St.11','St.12']
subcategories = ['GOM2021', 'GOM21FA', 'GOM22SP', 'GOM22SU']

# Define custom colors
color_map = {
    'GOM2021': 'darkred',
    'GOM21FA': 'blue',
    'GOM22SP': 'green',
    'GOM22SU': 'purple'
}

# Define custom labels (if they differ from subcategory names)
label_map = {
    'GOM2021': 'Summer 2021',
    'GOM21FA': 'Fall 2021',
    'GOM22SP': 'Spring 2022',
    'GOM22SU': 'Summer 2022'
}

barWidth = 0.25

fig, axs = plt.subplots(figsize=(10, 8)) 

# Set position of bars on x axis
r = np.arange(len(categories))
positions = [r + i*barWidth for i in range(len(subcategories))]

all_bars = []

for pos, subcat in zip(positions, subcategories):
    values = []
    for cat in categories:
        value = df[(df['Stn'] == cat) & (df['Camp'] == subcat)]['flux_aCDOM412'].values
        values.append(value[0] if len(value) > 0 else 0)
    bars = plt.bar(pos, values, width=barWidth, color=color_map[subcat], label=label_map[subcat])
    all_bars.append(bars)
    
plt.legend()

handles, _ = plt.gca().get_legend_handles_labels()
labels = [label_map[subcat] for subcat in subcategories]
plt.legend(handles[:len(subcategories)], labels)

# Add some text for labels, title, and custom x-axis tick labels
plt.xlabel('Stations: shallow to deep')
plt.ylabel(r'Flux of CDOM: $a_{\mathrm{CDOM}}$ 412 (nm) $(m^{-1})$ (m$^{-2}$ day$^{-1}$)')
plt.title('CDOM 412 flux: Benthic Chamber')
plt.xticks(r + barWidth, categories)
plt.ylim(-.1,.6)


plt.legend()
fig.set_facecolor('w')

plt.savefig("../Dissertation_projects/Figures/Flux_figures/CDOM412_flux_barplot.png")

plt.show()

In [ ]:
# Unique categories and subcategories
df = df_flux.dropna(subset=["flux_aCDOM350"])

categories = ['St.5B','St.4','St.MK','St.7','St.16','St.2','St.9','St.14','St.15','St.13','St.11','St.12']
subcategories = ['GOM2021', 'GOM21FA', 'GOM22SP', 'GOM22SU']

# Define custom colors
color_map = {
    'GOM2021': 'darkred',
    'GOM21FA': 'blue',
    'GOM22SP': 'green',
    'GOM22SU': 'purple'
}

# Define custom labels (if they differ from subcategory names)
label_map = {
    'GOM2021': 'Summer 2021',
    'GOM21FA': 'Fall 2021',
    'GOM22SP': 'Spring 2022',
    'GOM22SU': 'Summer 2022'
}

barWidth = 0.25

# Create figure with custom size
fig = plt.figure(figsize=(12, 10))
# Adjust the height ratios for the subplots
gs = gridspec.GridSpec(2, 1, height_ratios=[5, 1], hspace=0)  # Set hspace to 0

ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])

# Adjusted r for spacing between groups
r = np.arange(0, len(categories) * 1.5, 1.5)
positions = [r + i*barWidth for i in range(len(subcategories))]

# Adjusted code for broken axis and added spaces between the groups
for pos, subcat in zip(positions, subcategories):
    values = []
    errors = []
    for cat in categories:
        value = df[(df['Stn'] == cat) & (df['Camp'] == subcat)]['flux_aCDOM350'].values
        error = df[(df['Stn'] == cat) & (df['Camp'] == subcat)]['erros_aCDOM350'].values
        values.append(value[0] if len(value) > 0 else 0)
        errors.append(error[0] if len(error) > 0 else 0)
    
    ax1.bar(pos, [v if v > 0 else 0 for v in values], width=barWidth, color=color_map[subcat],
            label=label_map[subcat], yerr=[e if e > 0 else 0 for e in errors])
    ax2.bar(pos, [v if v < 0 else 0 for v in values], width=barWidth, color=color_map[subcat],
            label=label_map[subcat], yerr=[e if e > 0 else 0 for e in errors])

# Setting the limit for the break
ax1.set_ylim(0, .6)  
ax2.set_ylim(-.1, 0)

# Removing spines and ticks for the broken axis effect
ax1.spines['bottom'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax1.xaxis.tick_top()
ax1.tick_params(labeltop='off')
ax2.xaxis.tick_bottom()

# Add some text for labels, title, and custom x-axis tick labels
ax2.set_xlabel('Stations: shallow to deep')
ax1.set_ylabel(r'Flux of CDOM: $a_{\mathrm{CDOM}}$ 350 (nm) $(m^{-1})$ (m$^{-2}$ day$^{-1}$)')
ax1.set_title('CDOM 350 flux: Benthic Chamber')
ax2.set_xticks(r + 1.5*barWidth)  # Adjust for the spaces between groups
ax2.set_xticklabels(categories)

# Add black horizontal line at y=0
ax1.axhline(0, color="black", linewidth=0.5)
ax2.axhline(0, color="black", linewidth=0.5)

handles, _ = plt.gca().get_legend_handles_labels()
labels = [label_map[subcat] for subcat in subcategories]
ax1.legend(handles[:len(subcategories)], labels)

fig.set_facecolor('w')
plt.tight_layout()

plt.savefig("../Dissertation_projects/Figures/Flux_figures/CDOM350_flux_barplot.png")
plt.show()


In [ ]:
# Unique categories and subcategories
df = df_flux.dropna(subset=["flux_aCDOM350"])

categories = ['St.5B','St.4','St.MK','St.7','St.16','St.2','St.9','St.14','St.15','St.13','St.11','St.12']
subcategories = ['GOM2021', 'GOM21FA', 'GOM22SP', 'GOM22SU']

# Define custom colors
color_map = {
    'GOM2021': 'darkred',
    'GOM21FA': 'blue',
    'GOM22SP': 'green',
    'GOM22SU': 'purple'
}

# Define custom labels (if they differ from subcategory names)
label_map = {
    'GOM2021': 'Summer 2021',
    'GOM21FA': 'Fall 2021',
    'GOM22SP': 'Spring 2022',
    'GOM22SU': 'Summer 2022'
}

barWidth = 0.25

# Create figure with custom size
fig = plt.figure(figsize=(10, 5))
# Adjust the height ratios for the subplots
gs = gridspec.GridSpec(2, 1, height_ratios=[5, 1], hspace=0)  # Set hspace to 0

ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])

# Adjusted r for spacing between groups
r = np.arange(0, len(categories) * 1.5, 1.5)
positions = [r + i*barWidth for i in range(len(subcategories))]

# Adjusted code for broken axis and added spaces between the groups
for pos, subcat in zip(positions, subcategories):
    values = []
    errors = []
    for cat in categories:
        value = df[(df['Stn'] == cat) & (df['Camp'] == subcat)]['flux_aCDOM350'].values
        error = df[(df['Stn'] == cat) & (df['Camp'] == subcat)]['erros_aCDOM350'].values
        values.append(value[0] if len(value) > 0 else 0)
        errors.append(error[0] if len(error) > 0 else 0)
    
    ax1.bar(pos, [v if v > 0 else 0 for v in values], width=barWidth, color=color_map[subcat],
            label=label_map[subcat])#, yerr=[e if e > 0 else 0 for e in errors])
    ax2.bar(pos, [v if v < 0 else 0 for v in values], width=barWidth, color=color_map[subcat],
            label=label_map[subcat])#, yerr=[e if e > 0 else 0 for e in errors])

# Setting the limit for the break
ax1.set_ylim(0, .04)  
ax2.set_ylim(-.01, 0)

# Removing spines and ticks for the broken axis effect
ax1.spines['bottom'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax1.xaxis.tick_top()
ax1.tick_params(labeltop='off')
ax2.xaxis.tick_bottom()

# Add some text for labels, title, and custom x-axis tick labels
ax2.set_xlabel('Stations: shallow to deep')
ax1.set_ylabel(r'Flux of CDOM: $a_{\mathrm{CDOM}}$ 350 (nm) $(m^{-1})$ (m$^{-2}$ day$^{-1}$)')
ax1.set_title('CDOM 350 flux: Benthic Chamber')
ax2.set_xticks(r + 1.5*barWidth)  # Adjust for the spaces between groups
ax2.set_xticklabels(categories)

# Add black horizontal line at y=0
ax1.axhline(0, color="black", linewidth=0.5)
ax2.axhline(0, color="black", linewidth=0.5)

handles, _ = plt.gca().get_legend_handles_labels()
labels = [label_map[subcat] for subcat in subcategories]
ax1.legend(handles[:len(subcategories)], labels)

fig.set_facecolor('w')
plt.tight_layout()

plt.savefig("../Dissertation_projects/Figures/Flux_figures/_CDOM350_flux_barplot.png")
plt.show()


In [ ]:
lat_top=30
lat_bot=28
lon_left=-91.5
lon_rigth=-88.75
extent=[lon_left, lon_rigth, lat_bot, lat_top]

# set resolution of the features in map: '10m', '50m', or '110m'.  
# If you do this the first time it will take a while downloading the coastline 
# and other features. In the future it will be faster
res = '10m'

#Figure


def plot_mapbase(extent, res='10m', projection=ccrs.PlateCarree()):
    fig = plt.figure(figsize=(10,5)) # set fig size
    ax = fig.add_subplot(111, projection=projection)
    ax.set_extent(extent)
    ax.coastlines(res)
    ax.add_feature(cfeature.NaturalEarthFeature(category='physical', 
                                                name='land',
                                                facecolor='white', # change color here
                                                scale=res)) 
    ax.add_feature(cfeature.NaturalEarthFeature(category='physical', 
                                                name='rivers_lake_centerlines',
                                                edgecolor='blue', # change color here
                                                facecolor='none',
                                                scale=res))
    gl = ax.gridlines(xlocs=np.arange(lon_left, lon_rigth+1),
                      ylocs=np.arange(lat_bot, lat_top+1),
                      draw_labels=True, 
                      linewidth=0.5, color='k', linestyle=':')
    gl.top_labels = gl.right_labels = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    return fig, ax

fig, ax = plot_mapbase(extent)


In [ ]:
def plot_mapbase(extent, ax=None, res='10m', projection=ccrs.PlateCarree()):
    if ax is None:
        fig = plt.figure(figsize=(10,5)) # set fig size
        ax = fig.add_subplot(111, projection=projection)
        
    ax.set_extent(extent)
    ax.coastlines(res)
    ax.add_feature(cfeature.NaturalEarthFeature(category='physical', 
                                                name='land',
                                                facecolor='white', 
                                                scale=res)) 
    ax.add_feature(cfeature.NaturalEarthFeature(category='physical', 
                                                name='rivers_lake_centerlines',
                                                edgecolor='blue',
                                                facecolor='none',
                                                scale=res))
    gl = ax.gridlines(xlocs=np.arange(lon_left, lon_rigth+1),
                      ylocs=np.arange(lat_bot, lat_top+1),
                      draw_labels=True, 
                      linewidth=0.5, color='k', linestyle=':')
    gl.top_labels = gl.right_labels = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    return ax


In [ ]:
# List of unique Camp values you want to plot
camps = ['GOM2021', 'GOM21FA', 'GOM22SP', 'GOM22SU']

# Find global min and max for the colormap
vmin = -95
vmax = 95

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 12), subplot_kw={'projection': ccrs.PlateCarree()})
axes = axes.ravel()  # Flatten the 2x2 grid to access each subplot with a single index

for i, camp in enumerate(camps):
    df_subset = df_flux[df_flux['Camp'] == camp].dropna(subset=["flux_doc"])    
    
    # Plot the base map for each subplot
    plot_mapbase(extent, ax=axes[i])
    
    # Plotting
    mappable = axes[i].scatter(df_subset['Lon'], df_subset['Lat'],
                               c=df_subset['flux_doc'],
                               edgecolor='k',
                               cmap=cmo.curl,
                               s=200,
                               vmin=vmin,  # Set global min and max for colormap
                               vmax=vmax,
                               transform=ccrs.PlateCarree())
    
    # Set title for each subplot
    axes[i].set_title(camp)
    
# Add a single colorbar for the entire figure
cbar_ax = fig.add_axes([0.92, 0.3, 0.02, 0.4])
cb = fig.colorbar(mappable, cax=cbar_ax)
cb.set_label('DOC flux', size=10, color='black')
cb.ax.tick_params(labelcolor='black') 
cb.ax.tick_params(labelsize=10)

#plt.title(r'DOC flux:'+'\n'+' Benthic Chamber'+'\n')

fig.set_facecolor('w')

plt.savefig("../Dissertation_projects/Figures/Flux_figures/Map_DOC_Flux.png")

plt.show()


# List of unique Camp values you want to plot
camps = ['GOM2021', 'GOM21FA', 'GOM22SP', 'GOM22SU']

# Find global min and max for the colormap
vmin = -.6
vmax = .6

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 12), subplot_kw={'projection': ccrs.PlateCarree()})
axes = axes.ravel()  # Flatten the 2x2 grid to access each subplot with a single index

for i, camp in enumerate(camps):
    df_subset = df_flux[df_flux['Camp'] == camp].dropna(subset=["flux_aCDOM412"])
    
    # Plot the base map for each subplot
    plot_mapbase(extent, ax=axes[i])
    
    # Plotting
    mappable = axes[i].scatter(df_subset['Lon'], df_subset['Lat'],
                               c=df_subset['flux_aCDOM412'],
                               edgecolor='k',
                               cmap=cmo.diff,
                               s=200,
                               vmin=vmin,  # Set global min and max for colormap
                               vmax=vmax,
                               transform=ccrs.PlateCarree())
    
    # Set title for each subplot
    axes[i].set_title(camp)
    
# Add a single colorbar for the entire figure
cbar_ax = fig.add_axes([0.92, 0.3, 0.02, 0.4])
cb = fig.colorbar(mappable, cax=cbar_ax)
cb.set_label('a412 flux', size=10, color='black')
cb.ax.tick_params(labelcolor='black') 
cb.ax.tick_params(labelsize=10)

plt.title(r'CDOM 412 flux:'+'\n'+' Benthic Chamber'+'\n')

fig.set_facecolor('w')

plt.savefig("../Dissertation_projects/Figures/Flux_figures/Map_CDOM412_Flux.png")

plt.show()


In [ ]:
# List of unique Camp values you want to plot
camps = ['GOM2021', 'GOM21FA', 'GOM22SP', 'GOM22SU']

# Find global min and max for the colormap
vmin = -.04
vmax = .04

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 12), subplot_kw={'projection': ccrs.PlateCarree()})
axes = axes.ravel()  # Flatten the 2x2 grid to access each subplot with a single index

for i, camp in enumerate(camps):
    df_subset = df_flux[df_flux['Camp'] == camp].dropna(subset=["flux_aCDOM350"])
    
    # Plot the base map for each subplot
    plot_mapbase(extent, ax=axes[i])
    
    # Plotting
    mappable = axes[i].scatter(df_subset['Lon'], df_subset['Lat'],
                               c=df_subset['flux_aCDOM350'],
                               edgecolor='k',
                               cmap=cmo.diff,
                               s=200,
                               vmin=vmin,  # Set global min and max for colormap
                               vmax=vmax,
                               transform=ccrs.PlateCarree())
    
    # Set title for each subplot
    axes[i].set_title(camp)
    
# Add a single colorbar for the entire figure
cbar_ax = fig.add_axes([0.92, 0.3, 0.02, 0.4])
cb = fig.colorbar(mappable, cax=cbar_ax)
cb.set_label('a350 flux', size=10, color='black')
cb.ax.tick_params(labelcolor='black') 
cb.ax.tick_params(labelsize=10)

#plt.title(r'CDOM 350 flux:'+'\n'+' Benthic Chamber'+'\n')

fig.set_facecolor('w')

plt.savefig("../Dissertation_projects/Figures/Flux_figures/Map_CDOM350_Flux.png")

plt.show()


# Define the camps and flux types
camps = ['GOM2021', 'GOM21FA', 'GOM22SP', 'GOM22SU']
flux_types = ['flux_aCDOM350', 'flux_doc']

# Create a 3x4 subplot grid
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(15, 12), subplot_kw={'projection': ccrs.PlateCarree()})

for row, flux_type in enumerate(flux_types):
    # Determine the global min and max for the current flux type across all camps
    vmin = df_flux[df_flux['Camp'].isin(camps)][flux_type].min()
    vmax = df_flux[df_flux['Camp'].isin(camps)][flux_type].max()
    
    for col, camp in enumerate(camps):
        df_subset = df_flux[df_flux['Camp'] == camp].dropna(subset=[flux_type])
        
        mappable = axes[row, col].scatter(df_subset['Lon'], df_subset['Lat'],
                                          c=df_subset[flux_type],
                                          edgecolor='k',
                                          cmap=cmo.matter,
                                          s=100,
                                          vmin=vmin,
                                          vmax=vmax,
                                          transform=ccrs.PlateCarree())
        
        axes[row, col].set_title(f'{camp} - {flux_type}')
        
        plot_mapbase(extent, ax=axes[row, col])

    # Add a colorbar for each row (flux type)
    cbar_ax = fig.add_axes([0.92, 1-(0.29*(row+1)), 0.015, 0.125])
    cb = fig.colorbar(mappable, cax=cbar_ax)
    cb.set_label(flux_type, size=10, color='black')
    cb.ax.tick_params(labelcolor='black') 
    cb.ax.tick_params(labelsize=10)
    
fig.set_facecolor('w')

plt.savefig("../Dissertation_projects/Figures/Flux_figures/Maps_Flux.png")

plt.show()
